In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
from urllib.parse import urlparse, parse_qs
from shapely.geometry import Point
import time
import re
import os
import math

from blueribbon_crawling import BlueRibbonCrawler
from load_hotspots import load_hotspots
from haversine import haversine

In [27]:
class DatePopCrawler:
    def __init__(self, location, keyword, hotspots, is_food, crawl_new, blue_ribbon):
        self.location = location
        self.keyword = keyword
        self.search_word = location + " " + keyword
        self.hotspots = hotspots
        self.crawl_new = crawl_new
        self.blue_ribbon = blue_ribbon

        self.is_food = is_food

        self.data = pd.DataFrame(columns=['store_id', 'name', 'category', 'is_food', 'new_store', 'instagram_link', 'instagram_post', 'instagram_follower', 'visitor_review_count',
                                          'blog_review_count', 'distance_from_subway', 'on_tv', 'parking_available', 'no_kids', 'pet_available', 'seoul_michelin',
                                          'age-2030', 'gender-balance', 'on_blue_ribbon', 'image_urls', 'address'])
        self.empty_searchIframe = """//*[@id="_pcmap_list_scroll_container"]"""
        self.empty_entryIframe = """//*[@id="app-root"]"""
        self.empty_root = """//*[@id="root"]"""

        self.search_iframe = """//*[@id="searchIframe"]"""
        self.entry_iframe = """//*[@id="entryIframe"]"""

        self.store_dict = None

        self.driver = self.initialize_driver()
        self.wait = WebDriverWait(self.driver, 10)

    # 크롬 드라이버 설정
    def initialize_driver(self):
        options = webdriver.ChromeOptions()
        options.add_argument("--enable-logging")
        options.add_argument("--v=1")  # 로그 레벨 설정

        driver = webdriver.Chrome(options=options)
        driver.get("https://map.naver.com/")

        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[0])

        return driver

    # 한 매장의 정보를 저장하는 dictionary 변수 초기화
    def initialize_dictionary(self):
        self.store_dict = {
            "store_id": None,
            "name": "",
            "category": "",
            "is_food": self.is_food,
            "new_store": self.crawl_new,
            "instagram_link": None,
            "instagram_post": None,
            "instagram_follower": None,
            "hot_spot": False,
            "visitor_review_count": 0,
            "blog_review_count": 0,
            "distance_from_subway": None,
            "on_tv": False,
            "parking_available": False,
            "no_kids": False,
            "pet_available": False,
            "seoul_michelin": False,
            "age-2030": None,
            "gender-balance": None,
            "on_blue_ribbon": None,
            "image_urls": [],
            "address": None,
            "gps": {
                "latitude": None,
                "longitude": None,
            }
        }

    # 검색어를 input field에 입력 후 클릭
    def search_keyword(self):
        self.wait.until(EC.presence_of_element_located(
            (By.XPATH, self.empty_root)))

        css_selector = ".input_search"
        elem = self.wait.until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, css_selector)))

        elem.send_keys(self.search_word)
        time.sleep(1)
        elem.send_keys(Keys.RETURN)

    # "새로오픈" option 클릭
    def click_new_option(self):
        time.sleep(1)
        self.move_to_search_iframe()
        # "더보기" 버튼 클릭
        more_xpath = """//a[span[contains(text(),'더보기')]]"""
        more_button = self.wait.until(
            EC.element_to_be_clickable((By.XPATH, more_xpath)))
        self.driver.execute_script("arguments[0].click()", more_button)
        # "새로오픈" 버튼 클릭
        new_xpath = """//a[contains(text(),'새로오픈')]"""
        new_button = self.wait.until(
            EC.element_to_be_clickable((By.XPATH, new_xpath)))
        self.driver.execute_script("arguments[0].click()", new_button)

    # 한 매장에 대한 페이지 열기
    def get_into_store(self, index):
        # 매장의 정보를 저장할 저장할 dictionay 변수 초기화
        self.initialize_dictionary()
        # 매장 목록이 있는 search Iframe으로 이동
        self.move_to_search_iframe()

        # 현재 페이지의 index 번째 매장 클릭
        # store_xpath = f"""//*[@id="_pcmap_list_scroll_container"]/ul/li[{
        #     index}]/div[1]/a[1]"""
        store_xpath = f"""//*[@id="_pcmap_list_scroll_container"]/ul/li[{
            index}]//a[.//div[contains(@class, 'place_bluelink')]]"""

        elem = self.wait.until(
            EC.element_to_be_clickable((By.XPATH, store_xpath)))
        time.sleep(2)
        self.driver.execute_script("arguments[0].scrollIntoView(true);", elem)
        self.driver.execute_script("arguments[0].click()", elem)

        # entry Iframe에 접근하기위해 상위 frame으로 이동
        time.sleep(1)
        self.driver.switch_to.default_content()
        self.wait.until(EC.presence_of_element_located(
            (By.XPATH, self.empty_root)))
        self.driver.find_element(By.XPATH, self.empty_root)

        # 매장 정보가 있는 entry Iframe으로 이동
        self.wait.until(EC.presence_of_element_located(
            (By.XPATH, self.entry_iframe)))
        iframe_element = self.driver.find_element(By.XPATH, self.entry_iframe)
        # entry Iframe으로 실제로 이동하기 전에 해당 매장에 대한 url 얻기
        iframe_src = iframe_element.get_attribute('src')
        self.driver.switch_to.frame(iframe_element)
        self.driver.find_element(By.XPATH, self.empty_entryIframe)

        # 매장의 위도-경도 / 고유 ID 얻기
        parsed_url = urlparse(iframe_src)
        query_params = parse_qs(parsed_url.query)
        latitude = float(query_params.get('y')[0])
        longitude = float(query_params.get('x')[0])

        store_point = Point(longitude, latitude)
        self.store_dict["gps"] = {
            "latitude": latitude,
            "longitude": longitude
        }

        path_segments = parsed_url.path.split('/')
        store_id = path_segments[2]
        self.store_dict["store_id"] = store_id

        # 매장이 핫스팟에 위치해있는지 확인
        for i in range(len(self.hotspots)):
            polygon = self.hotspots[i]["polygon_area"]
            if polygon.contains(store_point):
                self.store_dict["hot_spot"] = True  # default = False
                break

        # entry Iframe에 "요청하신 페이지를 찾을 수 없습니다"라는 문구와 함께 매장 정보를 제대로 불러오지 못하는 경우가 드물게 발생
        # "새로고침" 버튼 클릭하여 매장 정보 다시 불러오기
        try:
            reset_elem = WebDriverWait(self.driver, 1).until(
                EC.presence_of_element_located((By.XPATH, """//a[contains(text(), '새로고침')]""")))

            self.driver.execute_script("arguments[0].click()", reset_elem)
            print("새로고침 발생")
        except NoSuchElementException:
            pass
        except Exception as e:
            print(f"새로고침 에러 {e}")

    # 대표사진 URL 추출 함수

    def get_image_urls(self, max_images=10, first_try=True):
        images_xpath = """/html/body/div[3]/div/div/div/div[6]/div[4]/div/div/div/div/a"""
        try:
            self.wait.until(EC.element_to_be_clickable(
                (By.XPATH, images_xpath)))
            images_xpath = images_xpath + "/img"
            image_elements = self.driver.find_elements(By.XPATH, images_xpath)
            image_urls = [img.get_attribute('src')
                          for img in image_elements][:max_images]
            return image_urls
        except StaleElementReferenceException:
            # 요소가 stale 상태인 경우
            if first_try:
                print("재시도")
                return self.get_image_urls(first_try=False)
            else:
                return []
        except TimeoutException as e:
            print("이미지 URL 추출 중 에러 발생: ", e)
            if first_try:
                self.move_to_tab("홈")
                self.move_to_tab("사진")
                return self.get_image_urls(first_try=False)
            else:
                return []

    def move_to_tab(self, tab_name):
        f"""//div[@role='tablist']/div/div[@class='flicking-viewport']/div[@class='flicking-camera']/a[span[text()={
            tab_name}]]"""
        # f"//a[.//span[contains(text(),{tab_name})]]"
        tab_xpath = f"""//div[@role='tablist']/div/div[@class='flicking-viewport']/div[@class='flicking-camera']/a[span[text()={
            tab_name}]]"""
        elem = self.driver.find_element(By.XPATH, tab_xpath)
        self.driver.execute_script("arguments[0].click()", elem)
        time.sleep(2)

    # 한 매장에 대한 정보 얻기
    def get_store_details(self):

        # 매장 정보 로딩을 위한 명시적 대기
        time.sleep(3)

        # 매장 이름, 카테고리
        try:
            store_name_xpath = """//*[@id="_title"]/div/span"""
            elem = self.wait.until(EC.presence_of_all_elements_located(
                (By.XPATH, store_name_xpath)))

            self.store_dict['name'] = elem[0].text
            self.store_dict['category'] = elem[1].text
        except TimeoutException as e:
            return False
        except Exception as e:
            print("매장 이름, 카테고리 에러: ", e)
            # 희귀하게 매장 이름을 크롤링하지 못하는 에러 발생
            # 해당 매장 생략하고 다음 매장 크롤링 진행
            return False

        # 신규 매장 여부
        # 일반 크롤링일 때만 실행
        if self.store_dict["new_store"] == False:
            try:
                new_open_xpath = """//*[@id='_title']/div/span[contains(text(), '새로오픈')]"""
                new_open_spans = self.driver.find_element(
                    By.XPATH, new_open_xpath)

                if new_open_spans:
                    self.store_dict["new_store"] = True
                else:
                    pass
            except NoSuchElementException:
                pass

        # 방문자 리뷰, 블로그 리뷰 개수
        try:
            elem_visitor = self.driver.find_element(
                By.XPATH, value="//a[contains(text(), '방문자리뷰')]")
            visitor_review_count = int(re.findall(
                r'\d+', elem_visitor.text.replace(",", ""))[0])
            self.store_dict['visitor_review_count'] = visitor_review_count
        except NoSuchElementException:
            self.store_dict['visitor_review_count'] = None
        try:
            elem_blog = self.driver.find_element(
                By.XPATH, value="//a[contains(text(), '블로그리뷰')]")
            blog_review_count = int(re.findall(
                r'\d+', elem_blog.text.replace(",", ""))[0])
            self.store_dict['blog_review_count'] = blog_review_count
        except NoSuchElementException:
            self.store_dict['blog_review_count'] = None

        # 도로명주소
        try:
            address_xpath = """//*[@id="app-root"]/div/div/div/div[5]/div/div[2]/div[1]/div/div[1]/div/a/span[1]"""
            address_elem = self.driver.find_element(By.XPATH, address_xpath)
            self.store_dict["address"] = address_elem.text
        except:
            print("도로명주소 에러")
            pass

        # 인스타그램 계정 존재 확인
        try:
            elem = self.driver.find_element(
                By.XPATH, value="//a[contains(@href, 'instagram.com')]")
            instagram_url = elem.get_attribute('href')

            wrong_url = ["https://instagram.com", "https://instagram.com/", "http://instagram.com",
                         "http://instagram.com/", "instagram.com", "instagram.com/",
                         "https://www.instagram.com/", "http://www.instagram.com/", "https://www.instagram.com",
                         "http://www.instagram.com"]
            if instagram_url in wrong_url:
                self.store_dict['instagram_link'] = None
            else:
                # query parameter 제거
                if "?" in instagram_url:
                    instagram_url = instagram_url.split("?")[0]
                # 계정 이름 이후 "/" 뒷부분 모두 제거
                if instagram_url.count("/") >= 4:
                    instagram_url = "/".join(instagram_url.split("/", 4)[:4])

                self.store_dict['instagram_link'] = instagram_url
        except (NoSuchElementException, TimeoutException):
            self.store_dict['instagram_link'] = None
            self.store_dict['instagram_post'] = None
            self.store_dict['instagram_follower'] = None
        except Exception as e:
            print("인스타그램 에러:", e)
            self.store_dict['instagram_link'] = None
            self.store_dict['instagram_post'] = None
            self.store_dict['instagram_follower'] = None

        # 서울 미쉐린 가이드 등재 여부
        try:
            michelin_xpath = """//div[a[contains(text(), '미쉐린 가이드 서울')]]"""
            self.driver.find_element(By.XPATH, michelin_xpath)
            self.store_dict['seoul_michelin'] = True
        except NoSuchElementException:
            self.store_dict['seoul_michelin'] = False
        except Exception as e:
            print("서울 미쉐린 가이드 에러:", e)
            self.store_dict['seoul_michelin'] = None

        # 지하철역 출구로부터 거리
        try:
            subway_xpath = "/html/body/div[3]/div/div/div/div[5]/div/div[2]/div[1]/div/div[1]/div/div"
            elem = self.driver.find_element(By.XPATH, subway_xpath)
            text = elem.text

            numbers = re.findall(r'\d+', text)
            if numbers:
                self.store_dict["distance_from_subway"] = numbers[-1]
        except NoSuchElementException:
            self.store_dict["distance_from_subway"] = None
        except Exception as e:
            print("지하철역 에러: ", e)
            self.store_dict["distance_from_subway"] = None

        # 방송 출연 여부
        try:
            tv_xpath = """//strong[descendant::span[text()='TV방송정보']]"""
            self.driver.find_element(By.XPATH, tv_xpath)
            self.store_dict['on_tv'] = True
        except NoSuchElementException:
            self.store_dict['on_tv'] = False
        except Exception as e:
            print("방송 출연 에러: ", e)
            self.store_dict['on_tv'] = None

        # 주차 가능, 반려동물 동반, 노키즈존
        try:
            convenient_xpath = "//strong[descendant::span[text()='편의']]/ancestor::div[1]/div/div"
            elem = self.driver.find_element(By.XPATH, convenient_xpath)
            convenients = elem.text

            for parking in ["주차", "발렛파킹"]:
                if parking in convenients:
                    self.store_dict["parking_available"] = True
                    break

            if "반려동물 동반" in convenients:
                self.store_dict["pet_available"] = True

            if "노키즈존" in convenients:
                self.store_dict["no_kids"] = True
        except NoSuchElementException:
            self.store_dict["parking_available"] = False
            self.store_dict["no_kids"] = False
            self.store_dict["pet_available"] = False
        except Exception as e:
            print("주차, 반려동물, 노키즈 에러: ", e)
            self.store_dict["parking_available"] = False
            self.store_dict["no_kids"] = False
            self.store_dict["pet_available"] = False

        # DataLab: 연령별 / 성별 검색 인기도
        try:
            # entryIframe 스크롤 끝까지 내려서 모든 컨텐츠 로딩
            last_height = self.driver.execute_script(
                "return document.body.scrollHeight")
            while True:
                self.driver.execute_script(
                    "window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(0.5)
                new_height = self.driver.execute_script(
                    "return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # DataLab 항목 찾고 해당 element로 스크롤 이동
            datalab_xpath = """//div[h2/span[contains(text(), '데이터랩')]]"""
            datalab_elem = self.driver.find_element(By.XPATH, datalab_xpath)
            self.driver.execute_script(
                "arguments[0].scrollIntoView(true);", datalab_elem)

            # "테마키워드"라는 text가 있는 경우, "더보기 버튼을 눌러줘야 연령별/성별 검색어 비율 확인 가능
            try:
                theme_keyword_xpath = """.//div/div/div/h3[contains(text(), '테마키워드')]"""
                datalab_elem.find_element(By.XPATH, theme_keyword_xpath)
                button_elem = datalab_elem.find_element(
                    By.XPATH, ".//div[2]/div/a")
                self.driver.execute_script("arguments[0].click()", button_elem)
            except NoSuchElementException:
                pass
            except Exception as e:
                print("데이터랩 더보기 버튼 에러: ", e)

            # 20대와 30대가 top 1, 2를 차지하는지 확인
            age_xpath = """//*[@id="bar_chart_container"]/ul/li/div[1]/span/span[1]"""
            age_elements = self.wait.until(
                EC.presence_of_all_elements_located((By.XPATH, age_xpath)))
            percentage_by_age = [
                round(float(item.text.replace('%', '')), 2) for item in age_elements]
            top_two = sorted(percentage_by_age, reverse=True)[:2]
            is_2030_in_top_two = percentage_by_age[1] in top_two and percentage_by_age[2] in top_two
            if is_2030_in_top_two:
                self.store_dict["age-2030"] = True
            else:
                self.store_dict["age-2030"] = False

            # 남성의 비율이 50%를 넘는지 확인
            gender_xpath = """//*[@id="pie_chart_container"]/div/*[local-name()='svg']/*[local-name()='g'][1]/*[local-name()='g'][3]/*[local-name()='g'][4]/*[local-name()='g']/*[local-name()='text'][2]"""
            gender_elements = self.wait.until(
                EC.presence_of_all_elements_located((By.XPATH, gender_xpath)))
            female, male = [round(float(item.text.replace("%", "")), 0)
                            for item in gender_elements]
            if male > 50:
                self.store_dict["gender-balance"] = False
            else:
                self.store_dict["gender-balance"] = True
        except NoSuchElementException:
            self.store_dict["age-2030"] = None
            self.store_dict["gender-balance"] = None
        except Exception as e:
            print("데이터랩 에러: ", e)
            self.store_dict["age-2030"] = None
            self.store_dict["gender-balance"] = None

        # 블루 리본 등재 여부 -> "맛집" 키워드일 경우에만 확인
        if self.is_food == True:
            if self.store_dict["name"].replace(" ", "") in [name.replace(" ", "") for name in self.blue_ribbon["name"].values]:
                indices = self.blue_ribbon.index[self.blue_ribbon["name"]
                                                 == self.store_dict["name"]].tolist()

                if len(indices) == 0:
                    indices = self.blue_ribbon.index[self.blue_ribbon["name"]
                                                     == self.store_dict["name"].replace(" ", "")].tolist()

                for i, index in enumerate(indices):
                    # 1. 도로명 주소 비교
                    # - 띄어쓰기 모두 제거한 상태로 비교
                    address1 = self.store_dict["address"].replace(" ", "")
                    address2 = self.blue_ribbon["address"][index].replace(
                        " ", "")

                    if address1 == address2:
                        print("도로명 주소 일치!!")
                        self.store_dict["on_blue_ribbon"] = True
                        break
                    # 2. 위도-경도 비교
                    lat1 = self.store_dict["gps"]["latitude"]
                    lon1 = self.store_dict["gps"]["longitude"]

                    lat2 = self.blue_ribbon["latitude"][index]
                    lon2 = self.blue_ribbon["longitude"][index]
                    distance = haversine(lat1, lon1, lat2, lon2)
                    if distance <= 50:
                        print("위도-경도 근사!!")
                        self.store_dict["on_blue_ribbon"] = True
                        break
                    if i + 1 == len(indices):
                        print("이름만 같았던 매장")
                        self.store_dict["on_blue_ribbon"] = False
                        break
            else:
                self.store_dict["on_blue_ribbon"] = False

        # 대표사진 크롤링
        try:
            imgtab_xpath = "//a[.//span[contains(text(),'사진')]]"
            elem = self.driver.find_element(By.XPATH, imgtab_xpath)
            self.driver.execute_script("arguments[0].click()", elem)
            time.sleep(2)  # 사진 로딩 대기

            self.store_dict["image_urls"] = self.get_image_urls()
        except NoSuchElementException as e:  # 사진 탭이 없는 경우
            print("사진 탭 없음")
            self.store_dict["image_urls"] = []

        # 인스타그램 크롤링(게시글 수, 팔로워 수)
        if self.store_dict['instagram_link'] != None:  # 인스타그램 계정이 있는 경우에만 실행
            try:
                instagram_embed_url = self.store_dict['instagram_link'] + "/embed"

                # 인스타그랩 탭으로 이동
                self.driver.switch_to.window(self.driver.window_handles[1])
                self.driver.get(instagram_embed_url)

                xpath = """//span[contains(., '팔로워') and contains(., '게시물')]/span/span"""
                elements = self.wait.until(
                    EC.presence_of_all_elements_located((By.XPATH, xpath)))

                follower = elements[0].text
                post = elements[1].text
                self.store_dict["instagram_follower"] = follower
                self.store_dict["instagram_post"] = post
            except (NoSuchElementException, TimeoutException):
                self.store_dict['instagram_link'] = None
                self.store_dict["instagram_follower"] = None
                self.store_dict["instagram_post"] = None
            # except Exception as e:
            #     print("인스타그램 에러: ", e)
            #     self.store_dict['instagram_link'] = None
            #     self.store_dict["instagram_follower"] = None
            #     self.store_dict["instagram_post"] = None

            # 네이버지도 탭으로 복귀
            self.driver.switch_to.window(self.driver.window_handles[0])

        print(self.store_dict)

        return True

    # 한 매장에 대한 크롤링 정볼르 DataFrame에 Insertion
    def insert_into_dataframe(self):
        new_data = pd.DataFrame([self.store_dict])
        self.data = pd.concat([self.data, new_data], ignore_index=True)

    # 한 페이지 크롤링
    def crawling_one_page(self):
        self.move_to_search_iframe()
        store_count = self.scroll_to_end()

        for i in range(1, store_count + 1):
            print(f"==== {i} 번째 매장 ====")
            self.get_into_store(index=i)
            if self.get_store_details():
                self.insert_into_dataframe()

    # 한 페이지에 대한 매장 개수 반환
    def scroll_to_end(self):
        li_xpath = """//*[@id="_pcmap_list_scroll_container"]/ul/li"""
        store_elements = self.wait.until(
            EC.presence_of_all_elements_located((By.XPATH, li_xpath)))
        store_count = len(store_elements)
        self.driver.execute_script(
            "arguments[0].scrollIntoView(true);", store_elements[-1])
        while True:
            time.sleep(0.5)
            store_elements = self.wait.until(
                EC.presence_of_all_elements_located((By.XPATH, li_xpath)))
            new_store_count = len(store_elements)

            if store_count == new_store_count:
                break
            store_count = new_store_count
            self.driver.execute_script(
                "arguments[0].scrollIntoView(true);", store_elements[-1])
        return store_count

    # searchIframe으로 이동
    def move_to_search_iframe(self):
        self.driver.switch_to.default_content()
        self.wait.until(EC.presence_of_element_located(
            (By.XPATH, self.empty_root)))
        self.wait.until(EC.frame_to_be_available_and_switch_to_it(
            (By.XPATH, self.search_iframe)))
        self.wait.until(EC.presence_of_element_located(
            (By.XPATH, self.empty_searchIframe)))

    # 다음 페이지로 이동
    def move_to_next_page(self):
        self.driver.switch_to.default_content()
        self.wait.until(EC.presence_of_element_located(
            (By.XPATH, self.empty_root)))
        self.wait.until(EC.frame_to_be_available_and_switch_to_it(
            (By.XPATH, self.search_iframe)))

        nextpage_xpath = """//a[span[contains(text(),'다음페이지')]]"""
        next_page_button = self.wait.until(
            EC.presence_of_element_located((By.XPATH, nextpage_xpath)))

        # 다음페이지 존재 여부 확인
        aria_disabled = next_page_button.get_attribute("aria-disabled")
        if aria_disabled == "true":
            return False
        else:
            next_page_button.click()
            time.sleep(2)
            return True

    def crawling(self):

        self.search_keyword()
        if self.crawl_new:
            self.click_new_option()
        for page in range(1, 7):
            print("="*10+f"page {page}" + "="*10)
            self.crawling_one_page()
            print(self.data)
            # 마지막 페이지인 경우
            if self.move_to_next_page() == False:
                break
            time.sleep(1)
        self.driver.quit()

In [28]:
if __name__ == "__main__":

    keywords = ["맛집", "공방", "만화카페", "스튜디오", "연극", "동물카페"]

    # 연극은 크롤링 지역 제한해야함
    # 추가로, ~~학원이나 ~~아카데미는 생략

    location = "성수동"
    keyword = "맛집"

    search_word = location + " " + keyword
    seoul_hotspots = load_hotspots('seoul_hotspots.csv')

    print(search_word, " 크롤링 시작")

    # 음식 크롤링 여부
    is_food = False
    blue_ribbon_data = None
    if keyword == "맛집":
        is_food = True
        blue_ribbon_crawler = BlueRibbonCrawler(location=location)
        blue_ribbon_crawler.crawling()
        blue_ribbon_data = blue_ribbon_crawler.data

    crawler = DatePopCrawler(location=location, keyword=keyword, hotspots=seoul_hotspots,
                             is_food=is_food, crawl_new=False, blue_ribbon=blue_ribbon_data)
    crawler.crawling()
    crawler_data_unique = crawler.data.drop_duplicates(
        subset='store_id', keep='first')

    if keyword == "맛집":

        print("Start to crawl new stores")
        # 동일 검색어로 신규오픈 매장 크롤링
        crawler_new = DatePopCrawler(location=location, keyword=keyword, hotspots=seoul_hotspots,
                                     is_food=is_food, crawl_new=True, blue_ribbon=blue_ribbon_data)
        crawler_new.crawling()
        crawler_new_data_unique = crawler_new.data.drop_duplicates(
            subset='store_id', keep='first')
    print(f"Crawling for {location} {keyword} is done.")

    if keyword == "맛집":
        merged_data = pd.merge(crawler_data_unique, crawler_new_data_unique,
                               on='store_id', how='outer', suffixes=('', '_new'))
    else:
        merged_data = crawler_data_unique

    print(f"merge하고 중복 제거 후 개수: {len(merged_data)}")

성수동 맛집  크롤링 시작
Blue Ribbon Survey crawling start......
Crawling page 0......


/Users/kimcheolhui/eagle-eye/eagle_eye/crawling/blueribbon_crawling.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.data = pd.concat([self.data, new_data], ignore_index=True)


Crawling page 1......
Crawling page 2......
Crawling page 3......
Crawling page 4......
Crawling page 5......
Crawling page 6......
Crawling page 7......
Blue Ribbon Survey crawling done!
==========page 1==========
==== 1 번째 매장 ====
{'store_id': '1088907629', 'name': '중화카츠 서울숲', 'category': '돈가스', 'is_food': True, 'new_store': False, 'instagram_link': 'https://www.instagram.com/joonghwa_kaz', 'instagram_post': '172', 'instagram_follower': '177', 'hot_spot': True, 'visitor_review_count': 826, 'blog_review_count': 399, 'distance_from_subway': '307', 'on_tv': False, 'parking_available': True, 'no_kids': False, 'pet_available': True, 'seoul_michelin': False, 'age-2030': False, 'gender-balance': True, 'on_blue_ribbon': False, 'image_urls': ['https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20201015_15%2F1602766331968ervak_JPEG%2FVqrcOb32WEoSgY8hnMow4urj.jpg', 'https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&s

KeyboardInterrupt: 